# Pumped-Storage Optimisation with Genetic Algorithm and MILP

In [6]:
import pandas as pd
import datetime
import numpy as np
import plotnine as pn
import plotly.graph_objs as go
import plotly.express as px
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import os
from itertools import product

# Import own implementations
from milp import MILP
import genetic_numpy
from genetic_numpy import GA_discrete_actions, evaluate_fitness

# Importing tuning libraries
import ray
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import ASHAScheduler

background_colour = "#F2F2F2"
pn.theme_set(
    pn.theme_classic()
    + pn.theme(
        text=pn.element_text(family="monospace"),
        plot_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        panel_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        legend_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
    )
)

%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


## Reading the Price data

In [7]:
df = (
    pd.read_csv("../01 - Data/spot_prices_utc.csv")
    .assign(utc_time=lambda x: pd.to_datetime(x.utc_time))
    .query('utc_time >= @pd.Timestamp("2022-08-01 00:00", tz="UTC")')
    .head(24 * 31)
    .reset_index(drop=True)
)
df

,spot,utc_time
0,387.62,2022-08-01 00:00:00+00:00
1,375.77,2022-08-01 01:00:00+00:00
2,372.42,2022-08-01 02:00:00+00:00
3,394.23,2022-08-01 03:00:00+00:00
4,430.72,2022-08-01 04:00:00+00:00
...,...,...
739,709.92,2022-08-31 19:00:00+00:00
740,679.91,2022-08-31 20:00:00+00:00
741,639.95,2022-08-31 21:00:00+00:00
742,569.92,2022-08-31 22:00:00+00:00


## The Power Plant

In [8]:
plant_params = {
    "EFFICIENCY": 0.75,
    "MAX_STORAGE_M3": 5000,
    "MIN_STORAGE_M3": 0,
    "TURBINE_POWER_MW": 100,
    "PUMP_POWER_MW": 100,
    "TURBINE_RATE_M3H": 500,
    "MIN_STORAGE_MWH": 0,
    "INITIAL_WATER_LEVEL_PCT": 0,
}
plant_params["INITIAL_WATER_LEVEL"] = (
    plant_params["INITIAL_WATER_LEVEL_PCT"] * plant_params["MAX_STORAGE_M3"]
)
plant_params["PUMP_RATE_M3H"] = (
    plant_params["TURBINE_RATE_M3H"] * plant_params["EFFICIENCY"]
)
plant_params["MAX_STORAGE_MWH"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * plant_params["TURBINE_POWER_MW"]

## GA

In [9]:
ga_solver = GA_discrete_actions(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"], actions_space= [-1, 0, 1]
)

In [10]:
analysis = ga_solver.tune(
    tune_config={
        "POP_SIZE": 500,
        "INITIAL_MUTATION_RATE": tune.uniform(0.05, 0.75),
        "FINAL_MUTATION_RATE": tune.uniform(0.001, 0.2),
        "INITIAL_EXPLORATION": tune.uniform(0.05, 0.95),
        "ELITISM": tune.uniform(0.05, 0.95),
        "SURVIVAL_RATE": tune.uniform(0.05, 0.95),
    },
    total_generations=500,
    timeout_s=60*60*10,
)

2024-04-13 21:49:20,519	INFO worker.py:1585 -- Calling ray.init() again after it has already been called.
2024-04-13 21:49:20,522	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
[I 2024-04-13 21:49:20,526] A new study created in memory with name: optuna


Trial name,fitness
train_0001a4a4,662976
train_00571e72,693367
train_0058d8c1,659190
train_0068eef9,651804
train_0077d4d8,697595
train_0077eaaf,648993
train_007baa38,471593
train_00880083,634584
train_00c8f82a,635668
train_00d76c27,5596


2024-04-13 21:50:03,894	WARNING util.py:202 -- The `callbacks.on_trial_result` operation took 1.052 s, which may be a performance bottleneck.
2024-04-13 21:50:03,897	WARNING util.py:202 -- The `process_trial_result` operation took 1.055 s, which may be a performance bottleneck.
2024-04-13 21:50:03,899	WARNING util.py:202 -- Processing trial results took 1.057 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2024-04-13 21:50:03,900	WARNING util.py:202 -- The `process_trial_result` operation took 1.058 s, which may be a performance bottleneck.
2024-04-13 21:50:32,813	WARNING util.py:202 -- The `callbacks.on_trial_result` operation took 2.435 s, which may be a performance bottleneck.
2024-04-13 21:50:32,815	WARNING util.py:202 -- The `process_trial_result` operation took 2.437 s, which may be a performance bottleneck.
2024-04-13 21:50:32,815	WARNING util.py:202 -- Processing trial results took 2.437 s, which may be a performance bott

In [11]:
analysis.best_config

{'POP_SIZE': 500,
 'INITIAL_MUTATION_RATE': 0.7070313584964933,
 'FINAL_MUTATION_RATE': 0.18927695720161727,
 'INITIAL_EXPLORATION': 0.9179428361908553,
 'ELITISM': 0.8441423442813326,
 'SURVIVAL_RATE': 0.4679128105055545}

In [12]:
tuning_history = (
    pd.concat(analysis.trial_dataframes, axis=0)
    .reset_index(drop=True)
    .filter(
        [
            "trial_id",
            "fitness",
            "training_iteration",
            "config/POP_SIZE",
            "config/INITIAL_MUTATION_RATE",
            "config/FINAL_MUTATION_RATE",
            "config/INITIAL_EXPLORATION",
            "config/ELITISM",
            "config/SURVIVAL_RATE",
        ]
    )
)

# Only keep the top combinations
num_best = 1000
top_combinations = tuning_history.groupby("trial_id").last().reset_index().sort_values("fitness", ascending=False).head(num_best)["trial_id"].to_list()
tuning_history = tuning_history.query("trial_id in @top_combinations")

In [13]:
(
    tuning_history
    .to_csv(
        f"./Tuning Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}_GA_Elite_numpy.csv",
        index=False,
    )
)